<div style="text-align: center;"><br>
<img src="https://assets-global.website-files.com/62b9d45fb3f64842a96c9686/62d84db4aeb2f6552f3a2f78_Quantinuum%20Logo__horizontal%20blue.svg" width="200" height="200" /></div>

# Developing Quantum Computational Workflows with InQuanto

This example notebook demonstrates some of the main modules in InQuanto&trade; to help researchers get started running Quantum Computational Chemistry calculations. 

Documentation for InQuanto is available at [inquanto.quantinuum.com](https://inquanto.quantinuum.com/).

## DEMO 1 - Applications scientists

### Using InQuanto to investigate *your* system 

- Use InQuanto extensions to interface with other classical chemistry codes to construct their systems
- For example, `inquanto-pyscf` allows interfacing with PySCF
- PySCF is written in Python and allows for native control and manipulation

Extensions Documentation: 
- InQuanto Extensions: https://inquanto.quantinuum.com/extensions/extensions-overview.html
- InQuanto PySCF Extension: https://inquanto.quantinuum.com/extensions/inquanto-pyscf.html
- InQuanto PySCF API documentation: https://inquanto.quantinuum.com/api/extensions/inquanto-pyscf_api.html

In [8]:
from inquanto.extensions.pyscf import ChemistryDriverPySCFMolecularROHF, ChemistryDriverPySCFMolecularRHF
from inquanto.geometries import GeometryMolecular

In [9]:
# Initialize the chemistry driver

# Could load an xyz or zmat file etc.
# mol or pbc 

# geometry = GeometryMolecular.load_xyz('phenyloxonium.xyz')

# Phenyloxonium
geometry_list= [
    ['O', (-1.13880,-1.89110, 0.11710)], 
    ['C', (-0.48870,-0.81700, 0.03930)],
    ['C', ( 0.92130,-0.83500, 0.02630)],
    ['C', (-1.16910, 0.41760,-0.02840)],
    ['C', ( 1.64540, 0.37090,-0.03660)],
    ['C', (-0.44530, 1.62320,-0.08100)],
    ['C', ( 0.96220, 1.60080,-0.08120)],
    ['H', ( 1.42890,-1.72290, 0.06010)],
    ['H', (-2.19170, 0.44570,-0.03190)],
    ['H', ( 2.66770, 0.35450,-0.04830)],
    ['H', (-0.94170, 2.51660,-0.11680)],
    ['H', ( 1.48760, 2.47710,-0.11410)],
    ['H', (-2.01490,-1.88740, 0.02920)],
    ['H', (-0.72290,-2.65290, 0.26620)]
]

geometry = GeometryMolecular(geometry_list)

# Control of lots of pyscf options here
driver = ChemistryDriverPySCFMolecularROHF(
    geometry=geometry,
    basis="sto-3g",
    point_group_symmetry = True,
    multiplicity = 1,
    charge = 1
)

In [10]:
# Can vizualize with another extension
from inquanto.extensions.nglview import VisualizerNGL

visualizer = VisualizerNGL(geometry)
visualizer.visualize_molecule(atom_labels="index")

NGLWidget()

In [11]:
# Get the data to be mapped using get_system
# This is the purpose of 'drivers'
# Here this performs Hartree-Fock and 
hamiltonian, fock_space, fock_state = driver.get_system()

In [12]:
# InQuanto classes and types
print(type(hamiltonian))
print(type(fock_space))
print(type(fock_state))

<class 'inquanto.operators._chemistry_integral_operator.ChemistryRestrictedIntegralOperator'>
<class 'inquanto.spaces._fermion_space.FermionSpace'>
<class 'inquanto.states._fermion_state.FermionState'>


In [13]:
# Can use helper methods to get classical benchmarks
print('HF:', driver.mf_energy)
print('MP2:', driver.run_mp2())
print('CCSD:', driver.run_ccsd()) # all electron
#print('CASCI:', driver.run_casci())

HF	 -302.0181424693229
MP2	 -302.39696594659375
CCSD	 -302.47209237724013


In [14]:
from inquanto.extensions.pyscf import AVAS

# Atomic valence active space
avas= AVAS(aolabels=['O 1s', 'O 2s', 'O 2p', '12 H 1s','13 H 1s' ], n_occ=2, n_vir=2, verbose=5)

avas_driver = ChemistryDriverPySCFMolecularROHF(
    geometry=geometry,
    basis='sto-3g',
    transf=avas,
    frozen=avas.frozenf,
    multiplicity=1,
    charge=1
)
avas_hamiltonian, avas_fock_space, avas_fock_state = avas_driver.get_system()


******** AVAS flags ********
aolabels = ['O 1s', 'O 2s', 'O 2p', '12 H 1s', '13 H 1s']
aolabels_vir = ['O 1s', 'O 2s', 'O 2p', '12 H 1s', '13 H 1s']
frozen = []
minao = minao
threshold = 0.2
threshold (virtual) = 0.2
with_iao = False
force_halves_active = False
canonicalize = True

** AVAS **
Total number of electrons: 50.0
  Total number of HF MOs  is equal to    42
  Number of occupied HF MOs is equal to  25
reference AO indices for minao ['O 1s', 'O 2s', 'O 2p', '12 H 1s', '13 H 1s']: [ 0  1  2  3  4 40 41]
(full basis) reference AO indices for sto-3g ['O 1s', 'O 2s', 'O 2p', '12 H 1s', '13 H 1s']: [ 0  1  2  3  4 40 41]
Active from occupied = 2 , eig [0.99494639 0.99961298]
Inactive from occupied = 23
Active from unoccupied = 2 , eig [0.98440564 0.99144623]
Inactive from unoccupied = 15
Number of active orbitals 4
# of alpha electrons 2
# of beta electrons 2


In [15]:
orbital_cubes = avas_driver.get_cube_orbitals(orbital_indices=[1,2])
orbitals = [visualizer.visualize_orbitals(orb, red_isolevel=-2.0, blue_isolevel=2.0) for orb in orbital_cubes]

In [16]:
orbitals[1]

NGLWidget()

In [17]:
# Can use helper methods to get classical benchmarks in an active space
print('HF\t', avas_driver.mf_energy)
print('"CAS" MP2\t', avas_driver.run_mp2())
print('"CAS" CCSD\t', avas_driver.run_ccsd()) # all electron
print('"CAS" CASCI\t', avas_driver.run_casci())

HF	 -302.0181424693229
"CAS" MP2	 -302.0306380425498
"CAS" CCSD	 -302.0215108455131
"CAS" CASCI	 -302.02151087635565


In [18]:
# inspect the AVAS Fock space
avas_fock_space.print_state(avas_fock_state)

 0 0a         :  1    
 1 0b         :  1    
 2 1a         :  1    
 3 1b         :  1    
 4 2a         :  0    
 5 2b         :  0    
 6 3a         :  0    
 7 3b         :  0    


- Many different `inquanto-pyscf` drivers for different embedding methods and environment models
    - QM/MM, COSMO, WFT-in-DFT,...
    - Also periodic systems
- Orbital transformation tools can be applied

In [19]:
# Express contains a compact variational quantum eigensolver method

from inquanto.ansatzes import FermionSpaceAnsatzUCCSD, FermionSpaceAnsatzUCCD
ansatz = FermionSpaceAnsatzUCCSD(fermion_space=avas_fock_space, fermion_state=avas_fock_state)
print(len(ansatz.free_symbols())) # number of UCCSD term parameters to optimize in VQE

26


In [20]:
from inquanto.operators import ChemistryRestrictedIntegralOperator
crio=ChemistryRestrictedIntegralOperator.from_FermionOperator(avas_hamiltonian.to_FermionOperator().compress(abs_tol=1e-4))
print(crio.df()[0:10])
q_ham = crio.qubit_encode()

   Coefficients            Terms
0   -247.289884                 
1    -27.564320          F0^ F0 
2      0.429683          F0^ F2 
3      0.262718          F0^ F4 
4     -0.000208          F0^ F6 
5      2.369955  F1^ F0^ F0  F1 
6      2.369955  F1^ F0^ F0  F1 
7     -0.208597  F1^ F0^ F0  F3 
8     -0.208597  F1^ F0^ F0  F3 
9     -0.132254  F1^ F0^ F0  F5 


In [21]:
# Statevector backend from pytket (no measurement circuits here)
from pytket.extensions.qiskit import AerStateBackend
sv_backend = AerStateBackend()

from inquanto.express import run_vqe
vqe = run_vqe(ansatz, q_ham, sv_backend)
print(vqe.final_value)
print(avas_driver.run_ccsd())
print(vqe.final_parameters)

# TIMER BLOCK-0 BEGINS AT 2024-05-07 10:37:39.396113
# TIMER BLOCK-0 ENDS - DURATION (s): 18.3732406 [0:00:18.373241]
-302.02150929414387
-302.0215108455131
{d0: -0.0005900034737191648, d1: 7.921038160772518e-07, d10: -1.0894312312150493e-08, d11: 2.5596186222059755e-08, d12: -3.649049488768865e-08, d13: -0.0006644520439024031, d14: -0.0281432592299454, d15: -1.2774961575494143e-07, d16: -1.277496167523326e-07, d17: -0.01298924488898081, d2: 2.559610434067746e-08, d3: -2.8646983556281982e-05, d4: -1.0894216528626738e-08, d5: 7.921038141233196e-07, d6: -0.0006006824888177041, d7: -3.6490410893782525e-08, d8: -0.0006644522191487454, d9: -2.86469817934077e-05, s0: 2.020225270495003e-05, s1: -9.469927055348106e-07, s2: 2.0202247771366247e-05, s3: -9.469925837966767e-07, s4: -0.001194551862935094, s5: 2.058534738517651e-05, s6: -0.0011945518614590366, s7: 2.0585347393190122e-05}


## DEMO 2 - Computational Chemists

In [23]:
# static and dynamic correlation tools
from inquanto.extensions.pyscf import FromActiveSpace
from inquanto.extensions.pyscf import ChemistryDriverPySCFMolecularRHF

# perform a native PySCF calculation
from pyscf import gto, scf, fci, mcscf

mf = gto.M(atom="Li 0 0 0; H 0 0 1.511", basis="cc-pvdz").apply(scf.RHF).run()
mc = mcscf.CASCI(mf, 4, 4)
mc.run()
(dm1, dm2, dm3, dm4) = fci.rdm.make_dm1234("FCI4pdm_kern_sf", mc.ci, mc.ci, 4, 4)


# load the pyscf calculation into an inquanto driver
driver_for_nev = ChemistryDriverPySCFMolecularRHF.from_mf(
    mf, frozen=FromActiveSpace(4, 4)(mf)
)

# explore convenience methods
nevpt_energy = driver_for_nev.get_nevpt2_correction(rdms=(dm1, dm2, dm3, dm4))

# from inquanto.computables.composite import RDM1234RealComputable

converged SCF energy = -7.98212617258416
CASCI E = -7.98224546483825  E(CI) = -9.03289512252191  S^2 = 0.0000000
Sr    (-1)',   E = -0.00137248954428
Si    (+1)',   E = -0.00000000000000
Sijrs (0)  ,   E = 0.00000000000000
Sijr  (+1) ,   E = -0.00000000000000
Srsi  (-1) ,   E = -0.00000000000000
Srs   (-2) ,   E = -0.01678088131772
Sij   (+2) ,   E = 0.00000000000000
Sir   (0)' ,   E = -0.00000000000000
Nevpt2 Energy = -0.018153370862006


Another way to bring your system to InQuanto:

Nexus's JupyterLab has NWChem preinstalled so we can download an example input (cell below) and obtain its integrals

In [25]:
!wget https://raw.githubusercontent.com/nwchemgit/nwchem/master/QA/tests/fcidump_hf/fcidump_hf.nw

--2024-05-07 10:38:29--  https://raw.githubusercontent.com/nwchemgit/nwchem/master/QA/tests/fcidump_hf/fcidump_hf.nw
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 689 [text/plain]
Saving to: ‘fcidump_hf.nw.2’

fcidump_hf.nw.2     100%[===================>]     689  --.-KB/s    in 0s      

2024-05-07 10:38:29 (48.2 MB/s) - ‘fcidump_hf.nw.2’ saved [689/689]



In [26]:
# use fcidump to interface with other codes
import subprocess
subprocess.run("nwchem " "fcidump_hf.nw" ">" "hf_fcidump.nwo", shell=True, check=True, executable='/bin/bash')

CompletedProcess(args='nwchem fcidump_hf.nw>hf_fcidump.nwo', returncode=0)

In [27]:
from inquanto.operators import ChemistryRestrictedIntegralOperator
from inquanto.spaces import FermionSpace
fcidump_ham= ChemistryRestrictedIntegralOperator.from_fcidump('hf_fcidump.fcidump')[0]
n_orbs=ChemistryRestrictedIntegralOperator.from_fcidump('hf_fcidump.fcidump')[1] # should be 12
n_elec=ChemistryRestrictedIntegralOperator.from_fcidump('hf_fcidump.fcidump')[2] # should be 10
multiplicity=ChemistryRestrictedIntegralOperator.from_fcidump('hf_fcidump.fcidump')[3] # should be 1 

In [28]:
fcidump_ham.df()[0:10]

,Coefficients,Terms
0,-71.436374,
1,-6.943249,F0^ F0
2,-0.152505,F0^ F2
3,0.641500,F0^ F8
4,-0.066312,F0^ F14
5,0.314453,F0^ F16
6,0.972105,F0^ F18
7,0.160611,F0^ F20
8,0.442139,F1^ F0^ F0 F1
9,0.442139,F1^ F0^ F0 F1


In [29]:
fcidump_ham.double_factorize()

- DMET, DMFT, Green's functions 
- Algorithms:  Variational Quantum Deflation, Quantum Subspace Expansion, Self Consistent Equation of Motion (contrast to e.g. EOM-CCSD)

## DEMO 3 - ‘Quantum quantum chemists’

### Computables (and nested computables)

In [32]:
# load in an example system from express
from inquanto.express import load_h5, get_system
#hehp
hehp_sto3g_data = load_h5("hehp_sto3g.h5", as_tuple=True)
ham, fock_space, fock_state=get_system(hehp_sto3g_data)

In [33]:
# this system comes with precalculated data
hehp_sto3g_data.energy_ccsd

-2.8510240323071567

In [34]:
# utilize a shallower form of UCCSD ansatz
from inquanto.ansatzes import FermionSpaceAnsatzChemicallyAwareUCCSD
CA_ansatz = FermionSpaceAnsatzChemicallyAwareUCCSD(fock_space, fock_state)

# can copy the state and change the symbol names so we have two similar states to test overlapping
CA_ansatz2 = CA_ansatz.copy()
CA_ansatz2.symbol_substitution(r"{}_1")

<inquanto.ansatzes._fock_space_ansatz_chemically_aware_uccsd.FermionSpaceAnsatzChemicallyAwareUCCSD, qubits=4, gates=99, symbols=3>

In [35]:
# use spaces to construct operators
spin_operator= fock_space.construct_spin_operator()

In [36]:
# and map those operators to qubits (assumes Jordan-Wigner here)
spin_operator.qubit_encode()

(0.75, ), (-0.375, Z0 Z1), (0.125, Y0 X1 Y2 X3), (-0.125, Y0 X1 X2 Y3), (0.125, X0 X1 Y2 Y3), (0.125, X0 X1 X2 X3), (0.125, Y0 Y1 Y2 Y3), (0.125, Y0 Y1 X2 X3), (-0.125, X0 Y1 Y2 X3), (0.125, X0 Y1 X2 Y3), (-0.375, Z2 Z3), (0.125, Z0 Z2), (-0.125, Z0 Z3), (-0.125, Z1 Z2), (0.125, Z1 Z3)

In [37]:
# other mappings are available
from inquanto.mappings import QubitMappingBravyiKitaev

bk_mapping = QubitMappingBravyiKitaev()
example_qo= bk_mapping.operator_map(ham, qubits=4)
# could also do bk_mapping.state_map

print(len(example_qo.items()))
print(type(example_qo))
example_qo.compress(abs_tol=1e-2)
print((len(example_qo.items())))

27
<class 'inquanto.operators._qubit_operator.QubitOperator'>
23


Fundamental building blocks

In [39]:
from inquanto.computables.atomic import ExpectationValue  # <A|\hat{O}|A>
from inquanto.computables.atomic import Overlap  # <A|\hat{O}|B>
from inquanto.computables.atomic import OverlapSquared # |<A|\hat{O}|B>|^2
from inquanto.computables.atomic import ExpectationValueDerivative # d_X #<A(X)|\hat{O}|A(X)>

In [40]:
expval = ExpectationValue(CA_ansatz, ham.qubit_encode())

gradient = ExpectationValueDerivative(CA_ansatz, ham.qubit_encode(), CA_ansatz.free_symbols_ordered())

overlap = Overlap(CA_ansatz, CA_ansatz2)

random_params = CA_ansatz.state_symbols.construct_from_array([0.01, 0.02, 0.03])

random_params2 = CA_ansatz2.state_symbols.construct_from_array([0.3, 0.2, -0.05])

# combine the two symboldicts into 1 new one
parameters = random_params.copy().update(random_params2)


In [41]:
# there are much more complex computables! e.g.
#from inquanto.computables.composite import KrylovSubspaceComputable

In [42]:
# Computables can be used to construct novel algorithms

from inquanto.algorithms import AlgorithmVQE
from inquanto.minimizers import MinimizerScipy

vqe = AlgorithmVQE(
        objective_expression=expval, #computable
        gradient_expression=gradient, # computable
        minimizer=MinimizerScipy(),
        initial_parameters=ansatz.state_symbols.construct_zeros())

# At this point this is kind of a symbolic algorithm
# The VQE represents minimizing the expval, using the gradient 
# A designer can build custom objective functions

In [43]:
# One way to evaluate these computable quantities is using statevector representations of 
# operators and states and performing the linear algebra directly.

from inquanto.protocols import SparseStatevectorProtocol
sv_protocol = SparseStatevectorProtocol(AerStateBackend())

# build makes the objects to evaluate the computables (here the sparse matrix representations)
vqe.build(protocol_objective=sv_protocol,
        protocol_gradient=sv_protocol)

In [44]:
# uses the protocols to evaluate the computables, minimizing the objective
vqe.run()
print(vqe.final_value)
h2_CA_params=vqe.final_parameters

# TIMER BLOCK-1 BEGINS AT 2024-05-07 10:38:46.701120
# TIMER BLOCK-1 ENDS - DURATION (s):  4.3999881 [0:00:04.399988]
-2.8510240299772174


# We haven't even thought about circuits yet!

In [45]:
# One way to evaluate an expectation value is through PauliAveraging \hat{O} = \sum_i h_i P_i 
# This gives us a 'protocol' for constructing circuits, measuring them, and interpreting the results

In [46]:
# Circuit based PROTOCOLS
from inquanto.protocols import PauliAveraging

from pytket.extensions.qiskit import AerBackend

from pytket.partition import PauliPartitionStrat

In [47]:
# instantiate an abstract measurement approach for quantities of this type
protocol = PauliAveraging(
   backend=AerBackend(), # need this for compilation
   shots_per_circuit=10000,
   pauli_partition_strategy=PauliPartitionStrat.CommutingSets
)

# build circuits using information in the computable
# this is where the computable unfolds and we remove redundancies
protocol.build_from(h2_CA_params, expval) #or random_params

# To measure this Hamiltonian on this state with CommutingSets we need 3 circuits
# without, it is 26
print("Number of circuits: ", protocol.n_circuit)

Number of circuits:  3


In [48]:
from pytket.circuit.display import render_circuit_jupyter

In [49]:
render_circuit_jupyter(protocol.get_circuits()[0])

In [50]:
# asynchronous, storeable, shareable, reuseable
# launch sends the 3 circuits for execution
handles=protocol.launch()
protocol.retrieve(handles)

In [51]:
#protocol.dataframe_measurements()

In [52]:
print('Stochastic 10000 shots noisefree evaluation: ', expval.evaluate(protocol.get_evaluator()), ' Ha')
#will converge to below in sampling limit

Stochastic 10000 shots noisefree evaluation:  -2.8513127495978305  Ha


In [53]:
print('Exact statevector evaluation ', expval.default_evaluate(h2_CA_params), ' Ha')

Exact statevector evaluation  -2.8510240299772174  Ha


In [54]:
# but what about noise? 

from inquanto.express import get_noisy_backend

# conveniance method for simple noise model
noisy_aer = get_noisy_backend(n_qubits=4, cx_err= 0.05, ro_err = 0.01)

# instantiate an abstract measurement approach for quantities of this type
protocol = PauliAveraging(
   backend = noisy_aer, 
   shots_per_circuit=10000,
   pauli_partition_strategy=PauliPartitionStrat.CommutingSets
)

protocol.build_from(h2_CA_params, expval) #or random_params

protocol.run()

protocol.evaluate_expectation_uvalue(CA_ansatz, ham.qubit_encode())
# result is far from SV VQE due to noise

-2.4455324386345985+/-0.008613651079363488

## H series usage and Nexus management


In [ ]:
# noise, circuit compilation (rebase)

In [56]:
# use Nexus to store and manage calculations
from pytket.extensions.nexus import Nexus, NexusBackend, QuantinuumConfig
nexus_client = Nexus()
nexus_client.login()

project_name = "InQuanto::Project"  # +'_'+str(time.strftime("%Y%m%d-%H%M%S"))
reuse_project=True
if reuse_project==True:
    my_nexus_project = nexus_client.get_project_by_name(project_name)
if reuse_project==False:
    my_nexus_project= nexus_client.new_project(project_name)

file_name = f"{my_nexus_project}-1A.json"

q_config = QuantinuumConfig(device_name='H1-1LE') #'LE' is noiseless local emulator

#could also be a pytket IBMQBackend or a Nexus IBMQConfig to target superconducting 

q_backend = NexusBackend(q_config, my_nexus_project)


Started using project with name: InQuanto::Project


In [57]:
protocol = PauliAveraging(
   backend = q_backend, 
   shots_per_circuit=1000,
   pauli_partition_strategy=PauliPartitionStrat.CommutingSets
)

from inquanto.spaces import QubitSpace
from inquanto.mappings import QubitMappingJordanWigner
stabilizers = QubitSpace(fock_space.n_orb * 2).symmetry_operators_z2_in_sector(
    ham.qubit_encode(), QubitMappingJordanWigner.state_map(fock_state)
)

# error mitigation tools
from inquanto.protocols import PMSV # also, SPAM and Qermit methods
pmsv = PMSV(stabilizers)

# Here we compile measurement circuits that can run on the H1-1 device using Nexus. 
# The circuit results can be used to give the expectation value of the Hamiltonian on 
# the ansatz state with optimized parameters, and discarding shots that violate symmetries 
# due to noise

protocol.build(h2_CA_params, CA_ansatz, ham.qubit_encode(), noise_mitigation=pmsv)

qhandles_pmsv=protocol.launch()
# go to nexus.quantinuum.com


/opt/conda/lib/python3.10/site-packages/inquanto/protocols/averaging/_mitigation.pye:213: UserWarning: The PMSV is still experimental.
  bJ'n`NF.>n[5[&0h)B:8:C,H`3Y5F_Q[i0H



In [58]:
render_circuit_jupyter(protocol.get_circuits()[0])

In [60]:
protocol.retrieve(qhandles_pmsv)

protocol.evaluate_expectation_uvalue(CA_ansatz, ham.qubit_encode())

-2.8484835242020816+/-0.007173161443587155

In [61]:
print(qhandles_pmsv)

[ResultHandle('fb8abb5f-146a-4af7-bd74-8e8b67a5bd5b', 2888845), ResultHandle('fb8abb5f-146a-4af7-bd74-8e8b67a5bd5b', 2888846), ResultHandle('fb8abb5f-146a-4af7-bd74-8e8b67a5bd5b', 2888847)]


- There are many ways to efficiently evaluate quantum chemical quantities with the computables and protocols structure and they have a lot of flexibility, useability, and efficiency built in
- Many different ansatz classes and tools to bring your ansatz
- Symmetry tools (tapering)
- Aim to support asynchronous workflows 

## DEMO 4 - Quantum computing, algorithms, or quantum information researchers

- Can construct a manual pytket circuit and use as an ansatz
- Concatenate unitaries
- Apply passes
- Build your own mappings

In [64]:
from inquanto.ansatzes import CircuitAnsatz

In [65]:
from pytket import Circuit, OpType

c = Circuit(4,2) # Create a circuit with 4 qubits and 2 classical bits
c.H(0) # Apply a gate to qubit 0
c.Rx(0.5,1) # Angles of rotation are expressed in half-turns (i.e. 0.5 means PI/2)

custom_ansatz = CircuitAnsatz(c)

#q_backend.default_compilation_pass(optimisation_level=0)

from pytket.passes import SequencePass, RemoveRedundancies, DecomposeMultiQubitsCX
seqpass = SequencePass([DecomposeMultiQubitsCX(), RemoveRedundancies()])

# using CA ansatz from before
recompiled_circ = CA_ansatz.to_CircuitAnsatz(compiler_pass=seqpass)
#can append circuits

In [66]:
recompiled_circ

<inquanto.ansatzes._circuit_ansatz.CircuitAnsatz, qubits=4, gates=99, symbols=3>

In [67]:
render_circuit_jupyter(recompiled_circ.state_circuit)

In [68]:
#from pytket.extensions.qiskit
#from pytket.extensions.quantinuum
#from pytket.extensions.cutensornet # TN
#takes care of rebasing, routing.. 

<div align="center"> &copy; 2024 by Quantinuum. All Rights Reserved. </div>